整体流程

1. 把你的内容拆成一块块的小文件块、对块进行了Embedding后放入向量库索引 （为后面提供语义搜索做准备）。

2. 搜索的时候把Query进行Embedding后通过语义检索找到最相似的K个Docs。

3. 把相关的Docs组装成Prompt的Context，基于相关内容进行QA，让GPT进行In Context Learning，用人话回答问题。

## 1. 环境准备

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "Key" # 填入自己的 OpenAI Key
os.environ["HTTP_PROXY"] = "127.0.0.1:1080"
os.environ["HTTPS_PROXY"] = "127.0.0.1:1080"

## 2. 载入 PDF

In [1]:
import PyPDF2

In [2]:
pdf_path = 'KOS：2023中国市场招聘趋势.pdf'
pdf_file = open(pdf_path, 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)

pdf 载入之后会分页进行保存，使用 extract_text() 提取文本

In [3]:
pdf_reader.pages[2].extract_text()

'前言\n2022年，依然是曲折起伏的一年。疫情政策调整、俄乌冲\n突、美联储加息等事件深刻影响着各行各业。\n回顾过去这一年，虽然市场充满了不确定性，却仍有不少\n企业在其中寻获了机遇。我们看到科技、消费品、电商等\n行业中，有不少企业开始将目光投向海外，扬帆起航，大\n力发展出海业务；机器人行业在智能技术的加持下，进入\n到了新的转型期，迎来跨跃式发展；新能源行业在国家政\n策的鼓励下，增长强劲，进入高质量增长阶段。\n几家欢喜，几家忧愁。也有行业在过去这一年中面临\n了前所未有的挑战。地产行业受疫情、政策调控等影\n响，发生了结构性变化；消费品行业受封控等影响，\n在逆境中求生，寻求新的突破机会点；医疗大健康行\n业在后疫情时代，行业泡沫被戳破，企业亟需找到创\n新破局之路；互联网行业中各个大厂纷纷开始优化结\n构，以守为攻。\n进入到2023年，国内疫情防控措施逐渐放松，\n并进入新阶段，这意味着国内消费将强劲反\n弹。同时，国家也在持续推进高水平对外开\n放，吸引外资。多重利好之下，各行业会有怎\n样的趋势？又将如何影响人才招聘市场？\nKOS高奥士国际特推出《2023中国人\n才招聘市场趋势》，分科技、医疗大\n健康、机器人、地产、新能源、消\n费/零售/电商、金融、法务、财务、\n人力资源十大板块总结2022年市场情\n况，展望2023年行业趋势，并根据市\n场动向为雇主品牌建设和人才求职方\n向提供了相关参考建议。\ni'

提取所有文本保存到 pdf_content 中

In [4]:
pdf_content = ''
for num in range(len(pdf_reader.pages)):
    page = pdf_reader.pages[num]
    pdf_content += page.extract_text()

In [5]:
len(pdf_content)

34400

## 3. 文本清洗

In [6]:
import re

### 3.1 去掉换行符

In [7]:
pdf_content = pdf_content.replace('\n', '') 
pdf_content = pdf_content.replace('\n\n', '') 
pdf_content = re.sub(r'\s+', ' ', pdf_content)

In [8]:
pdf_content[:500]

'中国 2023人才市场招聘趋势目录前言科技医疗大健康机器人地产新能源消费/零售/电商金融法务财务人力资源雇主建议人才建议关于我们i 01 152127394955636975838485前言2022年，依然是曲折起伏的一年。疫情政策调整、俄乌冲突、美联储加息等事件深刻影响着各行各业。回顾过去这一年，虽然市场充满了不确定性，却仍有不少企业在其中寻获了机遇。我们看到科技、消费品、电商等行业中，有不少企业开始将目光投向海外，扬帆起航，大力发展出海业务；机器人行业在智能技术的加持下，进入到了新的转型期，迎来跨跃式发展；新能源行业在国家政策的鼓励下，增长强劲，进入高质量增长阶段。几家欢喜，几家忧愁。也有行业在过去这一年中面临了前所未有的挑战。地产行业受疫情、政策调控等影响，发生了结构性变化；消费品行业受封控等影响，在逆境中求生，寻求新的突破机会点；医疗大健康行业在后疫情时代，行业泡沫被戳破，企业亟需找到创新破局之路；互联网行业中各个大厂纷纷开始优化结构，以守为攻。进入到2023年，国内疫情防控措施逐渐放松，并进入新阶段，这意味着国内消费将强劲反弹。同时，国家也在持续推进高水平对外开放，吸引外资。'

### 3.2 按句拆分

按照句子结束符号，将文档拆分成一句句话

In [9]:
pdf_sentences_mark = re.split('(；|。|！|\!|\.|？|\?)', pdf_content) 

In [10]:
pdf_sentences_mark[:10]

['中国 2023人才市场招聘趋势目录前言科技医疗大健康机器人地产新能源消费/零售/电商金融法务财务人力资源雇主建议人才建议关于我们i 01 152127394955636975838485前言2022年，依然是曲折起伏的一年',
 '。',
 '疫情政策调整、俄乌冲突、美联储加息等事件深刻影响着各行各业',
 '。',
 '回顾过去这一年，虽然市场充满了不确定性，却仍有不少企业在其中寻获了机遇',
 '。',
 '我们看到科技、消费品、电商等行业中，有不少企业开始将目光投向海外，扬帆起航，大力发展出海业务',
 '；',
 '机器人行业在智能技术的加持下，进入到了新的转型期，迎来跨跃式发展',
 '；']

In [11]:
pdf_sentences = []
for i in range(int(len(pdf_sentences_mark)/2)):
    sent = pdf_sentences_mark[2*i] + pdf_sentences_mark[2*i+1]
    pdf_sentences.append(sent)
if len(pdf_sentences_mark) % 2 == 1:
    pdf_sentences.append(pdf_sentences_mark[len(pdf_sentences_mark)-1])

In [12]:
pdf_sentences[:10]

['中国 2023人才市场招聘趋势目录前言科技医疗大健康机器人地产新能源消费/零售/电商金融法务财务人力资源雇主建议人才建议关于我们i 01 152127394955636975838485前言2022年，依然是曲折起伏的一年。',
 '疫情政策调整、俄乌冲突、美联储加息等事件深刻影响着各行各业。',
 '回顾过去这一年，虽然市场充满了不确定性，却仍有不少企业在其中寻获了机遇。',
 '我们看到科技、消费品、电商等行业中，有不少企业开始将目光投向海外，扬帆起航，大力发展出海业务；',
 '机器人行业在智能技术的加持下，进入到了新的转型期，迎来跨跃式发展；',
 '新能源行业在国家政策的鼓励下，增长强劲，进入高质量增长阶段。',
 '几家欢喜，几家忧愁。',
 '也有行业在过去这一年中面临了前所未有的挑战。',
 '地产行业受疫情、政策调控等影响，发生了结构性变化；',
 '消费品行业受封控等影响，在逆境中求生，寻求新的突破机会点；']

### 3.3 拆分段落

将多个句子拼成一个段落，每个段落最大长度为 300

In [13]:
paragraphs = []
max_len = 300
current_len = 0
current_para = ""

for sent in pdf_sentences:
    sent_len = len(sent)
    if current_len + sent_len <= max_len:
        current_para += sent
        current_len += sent_len
    else:
        paragraphs.append(current_para.strip())
        current_para = sent
        current_len = sent_len

paragraphs.append(current_para.strip())

In [14]:
paragraphs[:2]

['中国 2023人才市场招聘趋势目录前言科技医疗大健康机器人地产新能源消费/零售/电商金融法务财务人力资源雇主建议人才建议关于我们i 01 152127394955636975838485前言2022年，依然是曲折起伏的一年。疫情政策调整、俄乌冲突、美联储加息等事件深刻影响着各行各业。回顾过去这一年，虽然市场充满了不确定性，却仍有不少企业在其中寻获了机遇。我们看到科技、消费品、电商等行业中，有不少企业开始将目光投向海外，扬帆起航，大力发展出海业务；机器人行业在智能技术的加持下，进入到了新的转型期，迎来跨跃式发展；新能源行业在国家政策的鼓励下，增长强劲，进入高质量增长阶段。几家欢喜，几家忧愁。',
 '也有行业在过去这一年中面临了前所未有的挑战。地产行业受疫情、政策调控等影响，发生了结构性变化；消费品行业受封控等影响，在逆境中求生，寻求新的突破机会点；医疗大健康行业在后疫情时代，行业泡沫被戳破，企业亟需找到创新破局之路；互联网行业中各个大厂纷纷开始优化结构，以守为攻。进入到2023年，国内疫情防控措施逐渐放松，并进入新阶段，这意味着国内消费将强劲反弹。同时，国家也在持续推进高水平对外开放，吸引外资。多重利好之下，各行业会有怎样的趋势？又将如何影响人才招聘市场？']

## 4. 文档持久化

### 4.1 段落保存

将每个段落从一段字符串变成一个 Document，全篇 PDF 整合到 documents 中

In [15]:
from langchain.docstore.document import Document

In [16]:
documents = []
metadata = {"source": pdf_path}
for para in paragraphs:
    new_doc = Document(page_content=para, metadata=metadata)
    documents.append(new_doc)

In [17]:
documents[0]

Document(page_content='中国 2023人才市场招聘趋势目录前言科技医疗大健康机器人地产新能源消费/零售/电商金融法务财务人力资源雇主建议人才建议关于我们i 01 152127394955636975838485前言2022年，依然是曲折起伏的一年。疫情政策调整、俄乌冲突、美联储加息等事件深刻影响着各行各业。回顾过去这一年，虽然市场充满了不确定性，却仍有不少企业在其中寻获了机遇。我们看到科技、消费品、电商等行业中，有不少企业开始将目光投向海外，扬帆起航，大力发展出海业务；机器人行业在智能技术的加持下，进入到了新的转型期，迎来跨跃式发展；新能源行业在国家政策的鼓励下，增长强劲，进入高质量增长阶段。几家欢喜，几家忧愁。', metadata={'source': 'KOS：2023中国市场招聘趋势.pdf'})

### 4.2 使用 OpenAI Embeddings 进行文档向量化（二选一）

将每个 Document 变成一个 embedding，保存到向量数据库中（本地文件）

不同模型之间的目录需要分开，否则会出现写入文件的错误

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [22]:
persist_directory = 'db_openai'
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=documents, embedding=embedding, persist_directory=persist_directory)
vectordb.persist()

Using embedded DuckDB with persistence: data will be stored in: db_openai


In [23]:
vectordb = None

### 4.3 使用本地 Embeddings 进行文档向量化（二选一）

将每个 Document 变成一个 embedding，保存到向量数据库中（本地文件）

首先需要下载整个项目 https://huggingface.co/shibing624/text2vec-base-chinese

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

指定项目文件夹的路径

这里的报错 No sentence-transformers model found 不影响使用

In [2]:
embedding = HuggingFaceEmbeddings(model_name='text2vec-base-chinese')

/root/anaconda3/envs/chat/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name text2vec-base-chinese. Creating a new one with MEAN pooling.


这里需要选择数据库目录

In [20]:
persist_directory = 'db_huggingface'
vectordb = Chroma.from_documents(documents=documents, embedding=embedding, persist_directory=persist_directory)
vectordb.persist()

Using embedded DuckDB with persistence: data will be stored in: db_huggingface


In [21]:
vectordb = None

## 5. 文档读取

### 5.1 创建 retriever

retriever 用于根据索引查询向量数据库

搜索出 k 个最相关的 Document

In [3]:
vectordb = Chroma(persist_directory='db_huggingface', embedding_function=embedding)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

Using embedded DuckDB with persistence: data will be stored in: db_huggingface


### 5.2 创建 prompt 模板

创建一个 prompt 模板，用于进行文档问答

需要让语言模型根据文档内容来进行回答，超出范围的内容则回答我不知道

In [4]:
from langchain.prompts import PromptTemplate

In [5]:
prompt_template="""请注意：请谨慎评估Query与提示的Context信息的相关性，只根据本段输入文字信息的内容进行回答，如果Query与提供的材料无关，请回答"我不知道"，另外也不要回答无关答案：
    Context: {context}
    Query: {query}
    Answer:"""

prompt = PromptTemplate(
    input_variables=["context", "query"],
    template=prompt_template
)

### 5.3 编写提问

首先填写一个具体的问题

使用 retriever 从知识库中找到最相关的 k 个 Document

In [6]:
query = '2022年国内新能源车的渗透率是多少?'
docs = retriever.get_relevant_documents(query)

In [7]:
docs

[Document(page_content='到2035年，我国核电在运和在建装机容量将达到2亿千瓦左右，发电量约占全国发电量的10%左右。6. 氢燃料电池发展有望。虽然氢燃料电池还没有广泛应用在各能源行业，但国家已经部署加快能源领域关键核心技术和装备攻关，预计2023年国内将适度超前部署一批氢能项目，着力攻克可再生能源制氢和氢能储运、应用及燃料电池等核心技术，在氢能全产业链关键技术取得一定突破。46新能源各板块增长期望曲线图而新能源行业的2023年招聘趋势，也为上述论述所定调，各个板块都处于不同的增量预期曲线上（如上图）。', metadata={'source': 'KOS：2023中国市场招聘趋势.pdf'}),
 Document(page_content='2022年注定是新能源车及其产业链提前进入激烈竞争的一年， 既是技术产品营销的竞争，亦是人才争夺的竞争。这一年，国内新能源车的渗透率突破25%， 提前达到三年后目标。 KOS新能源招聘团队认为，在2022人才争夺战中，新能源车人才竞争的42三个战场会分别是研发、交付、营销。 各大车企招聘的增量直接反映其业务发展势头，直观例子是BYD在2022年的销量全面赶超特斯拉，而BYD在招聘市场上的热度亦然如此。我们看到人工智能、大数据技术、汽车研发、电池研发等研发相关岗位招聘需求上涨。制造工艺、机电工程、生产管理、供应链管理等交付类岗位始终需求强劲。', metadata={'source': 'KOS：2023中国市场招聘趋势.pdf'}),
 Document(page_content='销售人才依然是各大品牌争夺的重点，随着汽车保有量的上升，售后，客服，公关等岗。与新能源车相配套的电池、电机、电控、汽车电子、自动/辅助驾驶、车联网技术、换电站、充电桩、电池回收等相关的企业招聘需求持续走高。 两轮电动车的出海业务异常火热， 相关的产研、营销需求较高。2023New Energy432023年1月6日，国家能源局发布《新型电力系统发展蓝皮书（征求意见稿）》公开征求意见，望打造“新能源＋”模式，加快提升新能源可靠替代能力。意见稿提出推动多领域清洁能源电能替代，充分挖掘用户侧消纳新能源潜力。', metadata={'source': 'KOS：2023中国市场招聘趋势.pdf'})]

## 6. 文档问答

在上一步我们根据具体问题 query，得到了相关的段落 docs

接下来需要结合 prompt，让语言模型进行回答

### 6.1 引入 OpenAI 模型（二选一）

这里可以选择使用 OpenAI 模型或者本地的 ChatGLM 模型

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)

### 6.2 引入 ChatGLM 模型（二选一）

如果使用本地的 ChatGLM 模型，我们需要自己编写一个类

类的核心是 `_call` 函数，通过输入一段 prompt，返回一段输出结果

同时需要使用 `__init__` 对模型进行加载

这里没有给对话加入历史信息

In [10]:
from typing import Any, List, Mapping, Optional
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModel, AutoConfig

import torch


def torch_gc():
    # with torch.cuda.device(DEVICE):
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    

class ChatGLM(LLM):
    max_token: int = 10000
    temperature: float = 0.1
    top_p = 0.9
    tokenizer: object = None
    model: object = None

    def __init__(
        self,
        model_path: str = "chatglm-6b-int4",
        **kwargs
    ):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_path,
            trust_remote_code=True, 
            revision=model_path
        )
        model_config = AutoConfig.from_pretrained(
            model_path, 
            trust_remote_code=True, 
            revision=model_path
        )
        self.model = AutoModel.from_pretrained(
            model_path, 
            config=model_config, 
            trust_remote_code=True, 
            revision=model_path, 
            **kwargs
        ).half().cuda()
        self.model = self.model.eval()
        
    
    @property
    def _llm_type(self) -> str:
        return "ChatGLM"

    def _call(
        self, 
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        
        response, _ = self.model.chat(
            self.tokenizer,
            prompt,
            history=[],
            max_length=self.max_token,
            temperature=self.temperature,
        )
        torch_gc()
        return response

In [19]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [20]:
llm = ChatGLM()

Load kernel : chatglm-6b-int4/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 2
Parallel kernel is not recommended when parallel num < 4.
Using quantization cache
Applying quantization to glm layers


### 6.2 创建 load_qa_chain

LangChain 自带了 load_qa_chain，可以将文档和 prompt 编写到一起，然后调用模型，串联起整个流程

chain 的参数：

第一个是模型类型，这里使用的是默认的 gpt-3.5-turbo

第二个是传递类型，stuff 是将 docs 全部一次性传给模型进行归纳

第三个是前面编写好的模板，还未指定具体的 context 和 query

In [12]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)

### 6.3 调用 chain

填入具体参数，docs 是前面查找到的段落，query 是具体的问题

（这里有个问题是 context 变成了 input_documents，还不清楚二者是否对应）

调用 chain() 就会完成一次问答

In [13]:
result = chain({"input_documents": docs, "query": query}, return_only_outputs=True)

The dtype of attention mask (torch.int64) is not bool


In [14]:
result

{'output_text': '2022年国内新能源车的渗透率是25%。'}

### 6.4 尝试其他问题

In [15]:
query = "关于2022年的机器人行业企业密度，各国的排名怎么样？"
docs = retriever.get_relevant_documents(query)

In [16]:
docs

[Document(page_content='232023Robotics24国际机器人联合会（IFR）2022年底统计显示，中国机器人行业企业密度跻身世界第五位——前四位分别是，韩国、新加坡、日本、德国，美国则排在第九。2023年，机器人行业市场预计将保持20到25%的增长，而市场规模则仍在1000亿人民币以上。推动工业机器人发展有三大底层驱动因素，分别为人口结构改变、用劳用工成本的增加、下游应用产业的突出需求。而工业机器人产业链主要由零部件（上游）、机器人本体（中游）、系统集成（下游）以及行业应用等几个环节组成。', metadata={'source': 'KOS：2023中国市场招聘趋势.pdf'}),
 Document(page_content='去年，全球机器人市场规模达到513亿美元（折3576亿元人民币）。而国内机器人市场规模也突破了1213亿元人民币。在过往的5年里，国内机器人年均增长率为22%。2022年上半年机器人行业融资总额超315亿元人民币，其中，单笔融资金额达到20亿元人民币，其主要业务为轻量级协作机器人研发。国内很多机器人公司借助现有成熟工业链体系，完成弯道超车，在疫情期间，开始把业务重心移至国外。2022年，国内机器人公司出海业务比例占到30%，更有企业有超50%的业务来自海外市场。机器人赛道大体可分为三类：工业机器人、服务机器人、特种机器人。', metadata={'source': 'KOS：2023中国市场招聘趋势.pdf'}),
 Document(page_content='2022年注定是新能源车及其产业链提前进入激烈竞争的一年， 既是技术产品营销的竞争，亦是人才争夺的竞争。这一年，国内新能源车的渗透率突破25%， 提前达到三年后目标。 KOS新能源招聘团队认为，在2022人才争夺战中，新能源车人才竞争的42三个战场会分别是研发、交付、营销。 各大车企招聘的增量直接反映其业务发展势头，直观例子是BYD在2022年的销量全面赶超特斯拉，而BYD在招聘市场上的热度亦然如此。我们看到人工智能、大数据技术、汽车研发、电池研发等研发相关岗位招聘需求上涨。制造工艺、机电工程、生产管理、供应链管理等交付类岗位始终需求强劲。', metadata={'source': 'KOS：2023中国市场招聘趋势.pdf'})]

不需要重复创建 chain，只需要调用即可

In [17]:
result = chain.run(input_documents = docs, query = query)

In [18]:
result

'2022年底统计数据显示，中国机器人行业企业密度跻身世界第五位，前四位分别是韩国、新加坡、日本、德国。'